In [1]:
using Kuber, JSON
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


## Julia Workers in a ReplicaSet

Run a set of Julia processes, independent of each other, as a Kubernetes ReplicaSet.

Notice the Pod `spec` within the ReplicaSet `spec`. And notice the `replicas` attribute. That is how we specify how many replicas to run and of what.

Advantage of using a ReplicaSet, rather than a bunch of pods is that the whole set can be managed together. It can even be scaled up and down together.

In [2]:
julia_workers = kuber_obj(ctx, """{
           "kind": "ReplicaSet",
           "metadata":{
               "name": "julia-workers",
               "namespace": "default",
               "labels": {
                   "name": "julia-workers"
               }
           },
           "spec": {
               "replicas": 2,
               "selector": {
                   "matchLabels": {
                       "name": "julia-worker"
                   }    
               },
               "template": {
                   "metadata": {
                       "labels": {
                           "name": "julia-worker"
                       }
                   },
                   "spec": {
                       "containers": [{
                           "name": "julia",
                           "image": "julia:1.4.1",
                           "command": ["julia"],
                           "args": [
                               "-e",
                               "while true; println(time()); sleep(5); end"
                           ],
                           "resources": {
                               "limits": {
                                   "cpu": "1"
                               },
                               "requests": {
                                   "cpu": "0.5"
                               }
                           }
                       }]
                   }
               }
           }
       }""");

In [3]:
workers = put!(ctx, julia_workers);

In [4]:
;kubectl get pods

NAME                  READY   STATUS    RESTARTS   AGE
julia-workers-mvvsm   1/1     Running   0          7s
julia-workers-ttzgq   1/1     Running   0          7s


### Scale up. And scale back down.

We can scale the replica set up and down by updating it.

Which physical machine the pods will be launched on when scaled up, and which pods will be shut down when scaled down is determined by Kubernetes scheduler and the rules set for it.

To scale a replica set, we use the `update!` verb on it, specifying a patch to be applied.

Here we are scaling up the replica set from 2 pods to 3 pods.

In [6]:
using JSON
nworkers = 3
patch = JSON.parse("""[ { "op": "replace", "path": "/spec/replicas", "value": $nworkers } ]""");
JSON.print(patch, 4)

[
    {
        "value": 3,
        "op": "replace",
        "path": "/spec/replicas"
    }
]


In [7]:
update!(ctx, :ReplicaSet, "julia-workers", patch, "application/json-patch+json");

In [8]:
;kubectl get pods

NAME                  READY   STATUS    RESTARTS   AGE
julia-workers-mvvsm   1/1     Running   0          46s
julia-workers-ttzgq   1/1     Running   0          46s
julia-workers-xfntb   1/1     Running   0          8s


And here we are scaling down the replica set from 3 pods to 2 pods.

In [9]:
nworkers = 2
patch = JSON.parse("""[ { "op": "replace", "path": "/spec/replicas", "value": $nworkers } ]""")
update!(ctx, :ReplicaSet, "julia-workers", patch, "application/json-patch+json");

In [10]:
;kubectl get pods

NAME                  READY   STATUS    RESTARTS   AGE
julia-workers-mvvsm   1/1     Running   0          74s
julia-workers-ttzgq   1/1     Running   0          74s


In [11]:
delete!(ctx, workers);